In [1]:
## example using the pretrained convnet
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))

conv_base.summary()

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

# Two approaches

Running the convolutional base over your dataset will be fast, but won't allow data augmentation.

Or

Extending the model you have (conv_base) by adding Dense layers on top and running the whole thing end to end on the input data. This will allow data augmentation but is expensive.

## Fast feature extraction without data augmentation
Using the ImageDataGenerator to extract images as Numpy arrays as well as their labels.

In [2]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/Users/datascience4/Documents/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [3]:
print(validation_features.shape)

(1000, 4, 4, 512)


In [4]:
## feed them into a dense layer of 8192
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

print(validation_features.shape)

(1000, 8192)


In [5]:
## defining and training the densely connected classifier
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
             loss='binary_crossentropy',
             metrics=['acc'])

history = model.fit(train_features, train_labels,
                   epochs=30,
                   batch_size=20,
                   validation_data=(validation_features, validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 2s 832us/step - loss: 0.6479 - acc: 0.6370 - val_loss: 0.4650 - val_acc: 0.8070
Epoch 2/30
2000/2000 [==============================] - 1s 616us/step - loss: 0.4427 - acc: 0.8060 - val_loss: 0.3752 - val_acc: 0.8410
Epoch 3/30
2000/2000 [==============================] - 1s 614us/step - loss: 0.3626 - acc: 0.8440 - val_loss: 0.3361 - val_acc: 0.8550
Epoch 4/30
2000/2000 [==============================] - 1s 626us/step - loss: 0.3164 - acc: 0.8735 - val_loss: 0.3122 - val_acc: 0.8630
Epoch 5/30
2000/2000 [==============================] - 1s 643us/step - loss: 0.2901 - acc: 0.8790 - val_loss: 0.3026 - val_acc: 0.8690
Epoch 6/30
2000/2000 [==============================] - 1s 623us/step - loss: 0.2630 - acc: 0.8925 - val_loss: 0.2874 - val_acc: 0.8720
Epoch 7/30
2000/2000 [==============================] - 1s 634us/step - loss: 0.2481 - acc: 0.9000 - val_loss: 0.2812 - val_acc: 0.8800


In [6]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bH', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy pretrained')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bH', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss pretrained')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

## Still overfitting
The technique above is overfitting from the beginning. Therefore data augmentation would be useful on a small dataset like this but it would now become computationally expensive. Therefore a GPU is recommended from this point onwards.

You can the pretrained model to a Sequential model because they behave in a similar way to a layer.

In [7]:
## adding a densely connected classifier on top of the convolutional base
from keras import models, layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


## Freezing the pretrained layer
The pretrained dataset needs to be frozen to prevent the weights from being updated during training. Not doing may result in the dense layers on top (which are randomly initialised) updating weights which would then propagate through the network, virtually destorying the represenations previously learned.

In [8]:
print('This is the number of trainable weights '
     'before freezing the conv base: ', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights '
     'after freezing the conv base: ', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base:  30
This is the number of trainable weights after freezing the conv base:  4


Four weight tensors are now trainable two per layer (the main weight matrix, and the bias vector).

For the updates to take effect you must first compile the model. If you modify the weight trainability after compilation, you should then recompile the model or these changes will be ignored.

In [9]:
## training the model end to end with a frozzen convultional base
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) #don't augment validation

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
             metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 315s 3s/step - loss: 0.5831 - acc: 0.7035 - val_loss: 0.4551 - val_acc: 0.8000
Epoch 2/30
100/100 [==============================] - 322s 3s/step - loss: 0.4767 - acc: 0.7940 - val_loss: 0.3725 - val_acc: 0.8420
Epoch 3/30
100/100 [==============================] - 321s 3s/step - loss: 0.4303 - acc: 0.8065 - val_loss: 0.3317 - val_acc: 0.8590
Epoch 4/30
100/100 [==============================] - 316s 3s/step - loss: 0.3938 - acc: 0.8295 - val_loss: 0.3124 - val_acc: 0.8650
Epoch 5/30
100/100 [==============================] - 316s 3s/step - loss: 0.3932 - acc: 0.8300 - val_loss: 0.3113 - val_acc: 0.8630
Epoch 6/30
100/100 [==============================] - 316s 3s/step - loss: 0.3724 - acc: 0.8405 - val_loss: 0.2908 - val_acc: 0.8850
Epoch 7/30
100/100 [==============================] - 318s 3s/step - loss: 0.3463 - acc: 0.8585 - val_loss: 0.2821 - v